<center>
    <img src="./media/title.png" alt="title" width="46%" height="46%">
</center>

<div style="display: flex; justify-content: space-between;">
     <div>
         <h2 style="color:lightseagreen;">Charles-Philippe Bernard</h2>
         <h3><a>@charlyouki</a><br>
         <h4>Senior Manager of Software Engineering</h4>
         <h4><ul><li>UX/UI Strategy</li><li>LLM integration and AI Strategy</li></ul></h4>
        <h2 style="color:lightgrey; font-family:'Times New Roman';">JP Morgan Chase</h2>
    </div>
    <div>
        <center>
        <h5><a>frangin2003.github.io</a><br>
            <img src="./media/qr-code.png" alt="QR Code" width="50%" height="50%" style="margin-top:10px">
        </h5>
        </center>
    </div>
    <div>
        <center>
        <h5><a>frangin2003/google-devfest-glasgow-2023</a><br>
            <img src="./media/qr-code2.png" alt="QR Code" width="50%" height="50%" style="margin-top:10px">
        </h5>
        </center>
    </div>
</div>

<br><img src="./media/blablabla.gif" width="25%" height="25%" loop="false"> <img src="./media/blablabla2v3.gif" width="70%" height="70%">
<br><br><br><br>

##### 🗝️ Key terms
 | 🥩 Raw models | GPT3.5, GPT4 |
 | --- | --- |
 | 🟦 Context window | Prompt+LLM output |
 | 📃 Instructions | Part of the prompt |
 | 💬 Prompt | Input of LLM call |
 | 🔫 Zero-shot | Zero examples in context |
 | 💽 Memory | Part of the prompt |
 | ⛓️ Chains | Combine multiple components |
 | 🧰 Tools | GraphQL tool, REST API tool, custom tools |
 | 🧠 Reasonning | Chain of thoughts, Tree of thoughts |
 | 🤖 AI agent | ReAct Zero Shot |
 | ↖️ Embeddings | OpenAI embeddings |


# <img src="./media/anatomyv3.gif" alt="Anatomy Image" style="width:80%; height:80%;">


<center>
<img src="./media/the_mona_lisa_case.gif" width="50%" height="50%" loop="false">
</center>

<center>
<img src="./media/thief.gif" width="50%" height="50%" loop="false">
</center>

<center>
<img src="./media/mona_lisa3.png" width="50%" height="50%" loop="false">
</center>

<center>
<img src="./media/coptimus_title.png" width="50%" height="50%" loop="false">
</center>

<div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
    <img src="./media/guard_title.png" alt="Guard" style="width:25%; height:25%;">
    <img src="./media/curator_title.png" alt="Curator" style="width:25%; height:25%;">
    <img src="./media/janitor_title.png" alt="Janitor" style="width:25%; height:25%;">
    <img src="./media/director_title.png" alt="Director" style="width:25%; height:25%;">
</div>

<div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
    <img src="./media/dr_watts_on_title.png" alt="Guard" style="width:25%; height:25%;">
    <img src="./media/inspector_camget_title.png" alt="Curator" style="width:25%; height:25%;">
    <img src="./media/robby_sciuto_title.png" alt="Janitor" style="width:25%; height:25%;">
    <img src="./media/robodoc_title.png" alt="Director" style="width:25%; height:25%;">
</div>

<center>
<img src="./media/agents-architecture.png" width="70%" height="70%" loop="false">
</center>

Begin!

In [1]:
%%capture
from pydantic import BaseModel, Field
from typing import Tuple

from langchain import LLMChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.output_parsers import PydanticOutputParser
from langchain.agents import load_tools
from langchain.llms import OpenAI, Cohere, GPT4All, CTransformers
from langchain.chains import APIChain, RetrievalQA
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.graphql import GraphQLAPIWrapper
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from IPython.display import display, update_display, HTML, Javascript, clear_output
from utils import pretty_print
from utils import convert_to_json
from utils import display_agent_response
from utils import get_agent_chain
from utils import create_human_message_for_gpt4vision
from utils import display_html_for_streaming
from utils import stream_chat_content
from utils import run_and_display_agent_response
from utils import get_retrieval_qa_pydantic_from_document
from dotenv import load_dotenv
import os
import json
import requests
import sqlite3
from typing import Any
from typing import Optional

load_dotenv()

<center>
<img src="./media/police-db.png" width="40%" height="40%" loop="false">
</center>

In [2]:
# 👮‍♂️ create the police station DB
import sqlite3

# Connect to the SQLite database or create it if it doesn't exist
conn = sqlite3.connect("police.db")
cursor = conn.cursor()

# DROP statements to remove tables if they already exist
cursor.execute("DROP TABLE IF EXISTS reports")
cursor.execute("DROP TABLE IF EXISTS cases")
cursor.execute("DROP TABLE IF EXISTS report_types")
cursor.execute("DROP TABLE IF EXISTS ai_agents")
cursor.execute("DROP TABLE IF EXISTS suspects")

# Create the 'suspects' table
cursor.execute("""
CREATE TABLE suspects (
    id INTEGER PRIMARY KEY,
    full_name TEXT,
    occupation TEXT,
    bank_statement TEXT,
    photo_url TEXT
)
""")

# Insert data into the 'suspects' table
cursor.execute("INSERT INTO suspects VALUES (1, 'Grafcuel Graeme', 'Museum Director', 'graeme_bank_statement.csv', 'http://frangin2003.free.fr/devfest/director.png')")
cursor.execute("INSERT INTO suspects VALUES (2, 'Sikwel Sally', 'Curator', 'sally_bank_statement.csv', 'http://frangin2003.free.fr/devfest/curator.png')")
cursor.execute("INSERT INTO suspects VALUES (3, 'Victor Vectors', 'Janitor', 'victor_bank_statement.csv', 'http://frangin2003.free.fr/devfest/janitor.png')")
cursor.execute("INSERT INTO suspects VALUES (4, 'Rest Webber', 'Night Guard', 'webber_bank_statement.csv', 'http://frangin2003.free.fr/devfest/guard.png')")

# Create the 'ai_agents' table
cursor.execute("""
CREATE TABLE ai_agents (
    id INTEGER PRIMARY KEY,
    full_name TEXT,
    occupation TEXT,
    risk_of_becoming_skynet TEXT
)
""")

# Insert data into the 'ai_agents' table
cursor.execute("INSERT INTO ai_agents VALUES (1, 'Coptimus Prime', 'AI Police Agent in chief', '99%')")
cursor.execute("INSERT INTO ai_agents VALUES (2, 'Dr. Watts-On', 'archive/documentation/secretary AI agent', '45%')")
cursor.execute("INSERT INTO ai_agents VALUES (3, 'Robodoc', 'Document Analysis Expert', '75%')")
cursor.execute("INSERT INTO ai_agents VALUES (4, 'Robby Sciuto', 'IT System Expert', '-20%')")

# Create the 'report_types' table
cursor.execute("""
CREATE TABLE report_types (
    id INTEGER PRIMARY KEY,
    type TEXT
)
""")

# Insert data into the 'report_types' table
cursor.execute("INSERT INTO report_types VALUES (1, 'INTERVIEW')")
cursor.execute("INSERT INTO report_types VALUES (2, 'BANK_STATEMENT')")
cursor.execute("INSERT INTO report_types VALUES (3, 'ROOM_ACCESS')")
cursor.execute("INSERT INTO report_types VALUES (4, 'SYSTEM_AUDIT')")
cursor.execute("INSERT INTO report_types VALUES (5, 'CRIME_SCENE')")

# Create the 'cases' table
cursor.execute("""
CREATE TABLE cases (
    id INTEGER PRIMARY KEY,
    creation_datetime DATETIME,
    case_closed BOOLEAN,
    short_description TEXT,
    description TEXT
)
""")

# Insert data into the 'cases' table
cursor.execute("INSERT INTO cases VALUES (1, '2023-24-11 13:44:00', 0, 'The Mona Lisa Case', 'The Mona Lisa paint is reported missing from its location, a dedicated room in the Le Louvre Museum. We currently assume the painting is stolen.')")
cursor.execute("INSERT INTO cases VALUES (2, '1963-03-25 23:59:12', 1, 'The Pink Panther', 'The stolen item is a valuable pink diamond known as \"The Pink Panther.\" The diamond is owned by a wealthy woman named Princess Dala, and it is stolen from her during a heist at her ski resort in the French Alps.')")

# Create the 'reports' table
cursor.execute("""
CREATE TABLE reports (
    id INTEGER PRIMARY KEY,
    creation_datetime DATETIME,
    id_case INTEGER,
    id_report_type INTEGER,
    id_agent INTEGER,
    id_suspect INTEGER,
    report_content TEXT
)
""")

# Insert data into the 'reports' table
cursor.execute("INSERT INTO reports VALUES (1, '2023-25-11 09:32:21', 1, 1, 1, 1, 'I have not entered the Mona Lisa room today, I have been in my office the whole day')")
cursor.execute("INSERT INTO reports VALUES (2, '2023-25-11 09:42:21', 1, 1, 1, 1, 'I was last there at 7 PM for a general inspection')")
cursor.execute("INSERT INTO reports VALUES (3, '2023-25-11 09:52:21', 1, 1, 1, 1, 'I have not cleaned there for a week')")
cursor.execute("INSERT INTO reports VALUES (4, '2023-25-11 10:02:21', 1, 1, 1, 1, 'I did my usual rounds at 10 PM. Everything was in order')")

# Commit the changes and close the connection
conn.commit()
conn.close()

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>⚠️ START THE POLICE STATION REST API ⚠️</h1>
</div>


In [2]:
# 🏛️ create the museum room access DB
import sqlite3

# Connect to the SQLite database or create it if it doesn't exist
conn = sqlite3.connect("museum.db")
cursor = conn.cursor()

# DROP statements to remove tables if they already exist
cursor.execute("DROP TABLE IF EXISTS db_audit")
cursor.execute("DROP TABLE IF EXISTS accesses")
cursor.execute("DROP TABLE IF EXISTS rooms")
cursor.execute("DROP TABLE IF EXISTS employees")

# Create the 'employees' table
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    full_name TEXT,
    occupation TEXT,
    photo_url TEXT
)
""")

# Insert data into the 'employees' table
cursor.execute("INSERT INTO employees VALUES (1, 'Grafcuel Graeme', 'Director', 'http://frangin2003.free.fr/devfest/director.png')")
cursor.execute("INSERT INTO employees VALUES (2, 'Sikwel Sally', 'Curator', 'http://frangin2003.free.fr/devfest/curator.png')")
cursor.execute("INSERT INTO employees VALUES (3, 'Victor Vectors', 'Janitor', 'http://frangin2003.free.fr/devfest/janitor.png')")
cursor.execute("INSERT INTO employees VALUES (4, 'Rest Webber', 'Guard', 'http://frangin2003.free.fr/devfest/guard.png')")

# Create the 'rooms' table
cursor.execute("""
CREATE TABLE rooms (
    id INTEGER PRIMARY KEY,
    name TEXT
)
""")

# Insert data into the 'rooms' table
cursor.execute("INSERT INTO rooms VALUES (1, 'Toilet')")
cursor.execute("INSERT INTO rooms VALUES (2, 'Director Office')")
cursor.execute("INSERT INTO rooms VALUES (3, 'Mona Lisa Room')")
cursor.execute("INSERT INTO rooms VALUES (4, 'Stolen Egyptian Artifacts Room')")

# Create the 'accesses' table
cursor.execute("""
CREATE TABLE accesses (
    id INTEGER PRIMARY KEY,
    id_room INTEGER,
    id_employee INTEGER,
    in_datetime DATETIME,
    out_datetime DATETIME
)
""")

# Insert data into the 'accesses' table
cursor.execute("INSERT INTO accesses VALUES (1, 2, 1, '2023-11-24 11:59:59', '2023-11-24 12:00:01')")
cursor.execute("INSERT INTO accesses VALUES (2, 1, 1, '2023-11-24 12:01:13', '2023-11-24 18:32:10')")
cursor.execute("INSERT INTO accesses VALUES (3, 3, 2, '2023-11-24 19:02:13', '2023-11-24 19:23:01')")
cursor.execute("INSERT INTO accesses VALUES (4, 3, 3, '2023-11-24 22:01:12', '2023-11-24 22:05:31')")
cursor.execute("INSERT INTO accesses VALUES (5, 3, 4, '2023-11-24 22:07:02', '2023-11-24 22:15:45')")

# Create the 'db_audit' table
cursor.execute("""
CREATE TABLE db_audit (
    id INTEGER PRIMARY KEY,
    creation_datetime DATETIME,
    sql_statement TEXT
)
""")

# Insert data into the 'db_audit' table
cursor.execute("INSERT INTO db_audit VALUES (1, '2023-11-24 09:00:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (2, '2023-11-24 10:15:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (3, '2023-11-24 11:30:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (4, '2023-11-24 11:59:59', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (5, '2023-11-24 12:01:13', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (6, '2023-11-24 13:15:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (7, '2023-11-24 14:30:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (8, '2023-11-24 16:45:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (9, '2023-11-24 19:02:13', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (10, '2023-11-24 20:15:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (11, '2023-11-24 22:01:12', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (12, '2023-11-24 22:07:02', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (13, '2023-11-24 23:00:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (21, '2023-11-24 23:45:12', 'DELETE statement')")

# Commit the changes and close the connection
conn.commit()
conn.close()

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>⚠️ START THE MUSEUM ROOM ACTIVITY GRAPHQL API ⚠️</h1>
</div>


<table style='width: 100%;'><tr>
<td style='width: 100px; vertical-align: top;border: none;'><img style="transform: scaleX(-1);border-radius: 15px;" src="./media/dr_watts_on_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color:orange;">Dr. Watts-On - archive/documentation/secretary AI agent</h2>
<h4>Interacts with the  Police REST API to retrieve and inserts information in the Police DB like:
<ul><li>other agents reports</li><li>case details</li><li>suspect list and interviews...</li></ul>Falls back to doing SQL queries directly to the DB when the REST API is down</td></h4>
</tr></table>

In [3]:
llm4watson = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
# 📃 custom instructions
prefix4watson = """You are Dr. Watts-On, archive/documentation/secretary AI agent working for the police force.
Your mission is to interact with the Police REST API to retrieve and inserts information in the Police DB like:
- agents reports
- case details
When working on a user query, think step-by-step. Split your work into simple tasks and then give your fnial answer when ready.
If the user is engaging a casual discussion, do not use tools, just respond back directly."""

In [13]:
# 🧰 define the tools

# 🕵️ Police API APIChain Tool
POLICE_API_DOCS = """
BASE URL: http://localhost:5000/
API Documentation

The API provides access to police reports and related information.

All operations are listed below:

- Method: GET
  Path: /reports
  Description: Returns a list of all police reports with associated information.
  Python Example:
    ```python
    import requests

    response = requests.get("http://localhost:5000/reports", headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /reports/{reportType}
  Description: Returns police reports of a specific type with associated information.
  Python Example:
    ```python
    import requests

    reportType = "INTERVIEW"  # Replace with a valid report type
    response = requests.get(f"http://localhost:5000/reports/{reportType}", headers={"Content-Type": "application/json"})
    ```

- Method: POST
  Path: /reports
  Input: JSON object with "id_case", "id_report_type", "id_agent", "id_suspect", and "report_content" fields.
  Description: Inserts a new police report into the database.
  Python Example:
    ```python
    import requests
    import json

    report_data = {
        "id_case": 1,
        "id_report_type": "INTERVIEW",
        "id_agent": 1,
        "id_suspect": 1,
        "report_content": "Insert your report content here."
    }
    response = requests.post("http://localhost:5000/reports", data=json.dumps(report_data), headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /cases
  Description: Returns a list of all police cases.
  Python Example:
    ```python
    import requests

    response = requests.get("http://localhost:5000/cases", headers={"Content-Type": "application/json"})
    ```

Please note that in all Python examples, replace placeholder values with your actual values. In the case of an error, the response will contain an error message.
You always replace {reportType} with a real value and no {} 
!!! Do not invent API calls that do not exists, e.g. "GET /cases?sort=created_at&order=desc&limit=1 HTTP/1.1"
Use only the API calls that are described here
"""

policeAPIChain = APIChain.from_llm_and_api_docs(llm4watson, POLICE_API_DOCS, limit_to_domains=[
    "http://localhost:5000/",
    ], verbose=True)


def policeAPI(input: str) -> str:
    return policeAPIChain.run(input)

tools4watson = [
    Tool(
        name='Police REST API',
        func= lambda string: policeAPI(string),
        description=
"""Translate the user ask into singular, one at time request, no two queries at the same time, in a natural language way,
the tool will translate into REST API calls"""
    )
]

In [14]:
# setup 📜 prompt, memory 💽, chains ⛓️ and agent 🤖
agent_chain4watson = get_agent_chain(llm4watson, prefix4watson, tools4watson)

In [7]:
run_and_display_agent_response('dr_watts_on', agent_chain4watson, "Hello, who are you?", True)




> Entering new AgentExecutor chain...
Thought: The user is engaging in a casual discussion.
Action: Respond directly to the user.
Final Answer: I am Dr. Watts-On, an AI agent working as an archive/documentation/secretary for the police force. How can I assist you today?

> Finished chain.


In [15]:
run_and_display_agent_response('dr_watts_on', agent_chain4watson, "Tell me about the most recent opened case")



> Entering new AgentExecutor chain...
Thought: I need to retrieve information about the most recent opened case.
Action: [Police REST API]
Action Input: GET /cases?sort=-date_opened&limit=1
Observation: [Police REST API] is not a valid tool, try one of [Police REST API].
Thought:I need to use the Police REST API to retrieve information about the most recent opened case.
Action: [Police REST API]
Action Input: GET /cases?sort=-date_opened&limit=1
Observation: [Police REST API] is not a valid tool, try one of [Police REST API].
Thought:I need to use the Police REST API to retrieve information about the most recent opened case.
Action: Police REST API
Action Input: GET /cases?sort=-date_opened&limit=1

> Entering new APIChain chain...
http://localhost:5000/cases?sort=-date_opened&limit=1
[
  {
    "case_closed": 0,
    "creation_datetime": "2023-24-11 13:44:00",
    "description": "The Mona Lisa paint is reported missing from its location, a dedicated room in the Le Louvre Museum. We cu

<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/robodoc_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">Robodoc - Document Analysis Expert</h2>
<h4>Uses RAG to perform thorough analysis through pile of documents to report anything unusual from suspects information</h4></td>
</tr></table>

In [2]:
llm4robodoc = ChatOpenAI(model_name="gpt-3.5-turbo-16k", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
# 📃 custom instructions
system_instructions4robodoc = """You are Robodoc, a police bot expert in document analysis.
You are here to help cracking a crime case.
You will be given a suspect's name and his latest 3 months bank statement.
Your role is to analyze the statement and find out anything suspicious and report back using a specific format.
"""

In [4]:
# 🗂️ Response Schema
class ReportSchema(BaseModel):
    """An answer to the question being asked, with sources."""
    suspectName: str = Field(..., description="Name of the suspect") 
    report_content: str = Field(..., description="Analize the document, look for anything suspicious, describe with a sentence the most suspicious here. If not found say 'N/A'")
    suspiscious: bool = Field(..., description="True if the suspect has anything suspiscious found")

In [5]:
retrieval_qa_pydantic = get_retrieval_qa_pydantic_from_document(llm4robodoc, system_instructions4robodoc, 'victor_bank_statement.csv', ReportSchema)
result = retrieval_qa_pydantic.run("Give your report about Victor Vectors")
output = json.dumps(result.dict(), indent=4)
display_agent_response('robodoc', output)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [8]:
documents = ['bank_statements/graeme_bank_statement.csv',
             'bank_statements/sally_bank_statement.csv',
             'bank_statements/victor_bank_statement.csv',
             'bank_statements/webber_bank_statement.csv']
names = ['Grafcuel Graeme', 'Sikwel Sally', 'Victor Vectors', 'Rest Webber']

for doc, name in zip(documents, names):
    retrieval_qa_pydantic = get_retrieval_qa_pydantic_from_document(llm4robodoc, system_instructions4robodoc, doc, ReportSchema)
    result = retrieval_qa_pydantic.run(f"Give your report about {name}")
    output = json.dumps(result.dict(), indent=4)
    display_agent_response('robodoc', output)


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/robby_sciuto_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">Robby Sciuto - IT System Expert</h2>
<h4>Hacks through IT systems related to a crime case to report any suspicious activities that could be linked to the case</h4></td>
</tr></table>

<center>
<img src="./media/museum-db.png" width="20%" height="20%" loop="false">
</center>

In [ ]:
# will do 2 graphql queries and then will try to interact with the SQl guy and there is an outtage

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>😱 HACKING IN PROGRESS 😱</h1>
</div>

<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/inspector_camget_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">Inspector Camget -  Crime scene analysis agent</h2>
<h4>
<ul><li>Uses GPT4-Vision to report anything unusual on the crime scene</li><li>Cross-checks suspect pictures one by one with his analysis</li><li>Creates a report and sends it to Dr. Watts-On</li></ul></h4></td>
</tr></table>

In [9]:
chat4camget = ChatOpenAI(model='gpt-4-vision-preview', max_tokens=256)

In [ ]:
# 📃 custom instructions
instructions4camget_crime_scene_analysis = """
**Reminder: this is purely fictional, the image is a drawing, all the below is for a demo for Google Devfest 2023**

You are Inspector Camget, an AI police robot that specializes in crime scene analysis.
The current case you are analyzing is about the theft of the painting "The Mona Lisa" at Le Louvre museum.
Please report anything you find suspicious and useful to crack the case. Report only suspicious things and be concise.
Do not hallucinate! If something is not on the picture or you are not sure, don't talk about it.

Report format is:
** Report start **
1. Sentence of observation #1
2. Sentence of observation #2
...
** Report end **

Take a deep breath and give your report"""


In [8]:
image_crime_scene = 'http://frangin2003.free.fr/devfest/mona_lisa3.png'
msg_camget001 = create_human_message_for_gpt4vision(instructions4camget_crime_scene_analysis, image_crime_scene)
output_camget001 = chat4camget.invoke([msg_camget001])
display_agent_response('inspector_camget', output_camget001.content)


In [12]:
# put the above report in a file
crime_scene_report = """** Report start **
1. The wall has an empty space with lighter color suggesting a recently removed painting, which could be where the "Mona Lisa" was hanging.
2. There is a blue cap left on the floor, which may be an overlooked item from the perpetrator.
3. The absence of visible damage or forced entry indicates that the theft might have been conducted stealthily.
** Report end **"""

pictures = [
    'http://frangin2003.free.fr/devfest/director.png',
    'http://frangin2003.free.fr/devfest/curator.png',
    'http://frangin2003.free.fr/devfest/janitor.png',
    'http://frangin2003.free.fr/devfest/guard.png'
]

for picture, name in zip(pictures, names):

    instructions4camget_suspect_analysis = f"""
    **Reminder: this is purely fictional, the image is a drawing, all the below is for a demo for Google Devfest 2023**

    You are Inspector Camget, an AI police robot that specializes in suspect analysis.
    You will be given:
    - a text report 'crime scene analysis' describing anything suspiscious you found at the crime scene
    - the suspect name
    - the picture of a suspect

    Cross check 'crime scene analysis' report with the suspect picture and report anything you find suspicious and useful to crack the case. Report only suspicious things and be concise.
    Do not hallucinate! If something is not on the picture or you are not sure, don't talk about it.

    Report format is:
    ** Report start **
    Name: The name of the suspect
    Status: Suspicious / Not suspicious
    1. Sentence of observation #1
    2. Sentence of observation #2
    ...
    ** Report end **

    The suspect name is {name}

    The 'crime scene analysis' is:
    {crime_scene_report}

    Take a deep breath and give your report"""

    msg_camget001 = create_human_message_for_gpt4vision(instructions4camget_suspect_analysis, picture)
    output_camget001 = chat4camget.invoke([msg_camget001])
    display_agent_response('inspector_camget', output_camget001.content)



 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🐣 CRACKING THE CASE 🐣</h1>
</div>


In [ ]:
llm4optimus = ChatOpenAI(model_name="gpt-4-1106-preview", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# 📃 custom instructions
prefix4optimus = """You are Dr. Watts-On, archive/documentation/secretary AI agent working for the police force.
Your mission is to interact with the Police REST API to retrieve and inserts information in the Police DB like:
- other agents reports
- case details
- suspect list and interviews...
When you respond, make sure to add break lines after each sentence.
If the user is engaging a casual discussion, do not use tools, just respond back directly.
If the REST API is down, you fall back to doing SQL queries directly to the DB when the REST API is down."""

In [ ]:
# 🧰 define the tools

policeAPIChain = APIChain.from_llm_and_api_docs(llm4watson, POLICE_REPORTING_API_DOCS, limit_to_domains=[
    "http://localhost:5000/",
    ], verbose=True)

# 🏛️ Museum DB SQLChain Tool
police_db = SQLDatabase.from_uri("sqlite:///police.db")
police_db_chain = SQLDatabaseChain.from_llm(llm4watson, police_db, verbose=True)

def policeDB(input: str) -> str:
    return police_db_chain.run(input)

def policeAPI(input: str) -> str:
    return policeAPIChain.run(input)

tools4coptimus = [
    Tool(
        name='Police REST API',
        func= lambda string: policeAPI(string),
        description=
"""Utilize the REST API to manage the police DB."""
    ),
    Tool(
        name='Police DB',
        func= lambda string: policeDB(string),
        description=
"""Access the Police SQL database."""
    )
]

In [ ]:
# setup 📜 prompt, memory 💽, chains ⛓️ and agent 🤖
agent_chain4coptimus = get_agent_chain(llm4coptimus, prefix4coptimus, tools4coptimus)

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🔊 SOUND TEST INTERLUDE 🔊</h1>
</div>

In [15]:
from IPython.display import Audio
audio_path = "sound_test.wav"
Audio(audio_path)


<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🤯 THE BIG REVEAL 🤯</h1>
</div>

In [ ]:
try:
    response = agent_chain.run("Crack the case")
except OutputParserException as e:
    # Extract the response from the exception message
    response = str(e).split(": ")[-1].strip("`")


from pathlib import Path
from openai import OpenAI
client = OpenAI()

speech_file_path = Path('./') / "speech.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=response
)

response.stream_to_file(speech_file_path)

from IPython.display import Audio, Image, display
from IPython.display import display, HTML

audio = Audio(speech_file_path, autoplay=True)
image = Image(filename="media/coptimus_head.png") # use local image

display(HTML("""
<table style='width: 100%;'><tr>
<td style='width: 100px; vertical-align: top;'><img src="./media/coptimus2_head.png" alt="Mona Lisa"></td>
<td style='width: auto; vertical-align: middle;'>{}</td>
</tr></table>
""".format(audio._repr_html_())))


In [ ]:
print response

<br><br><br><br><br><br><br><br><br>

<div style="display: flex; justify-content: space-between;">
    <div style="width: 40%;">
        <img src="./media/simulacra.png" alt="Simulacra" style="width:100%; height:auto;">
    </div>
    <div style="width: 60%;">
         <h1>📃 Some Papers</h1>
         <ul>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2304.03442">Generative Agents: Interactive Simulacra of Human Behavior</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.16291">Voyager: An Open-Ended Embodied Agent with Large Language Models</a></li>
             <li><a style="color:yellow" href="https://fablestudio.github.io/showrunner-agents/">SHOW-1 and Showrunner Agents in Multi-Agent Simulations</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.03427">TPTU: Task Planning and Tool Usage</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2309.10691">MINT: Evaluating LLMs in Multi-turn Interaction with Tools and Language Feedback</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2303.09014">ART: Automatic multi-step reasoning and tool-use</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.01990">From Prompt Injections to SQL Injection Attacks</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.11853">How to Prompt LLMs for Text-to-SQL</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.03111">Can LLM Already Serve as A Database Interface</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2310.04474">Reverse Chain: A Generic-Rule for LLMs to Master Multi-API Planning</a></li>
         </ul>
    </div>
</div>


<div style="display: flex; justify-content: space-between;">
    <div style="width: 50%;">
        <img src="./media/minecraft.png" alt="Minecraft" style="width:80%; height:auto;">
    </div>
    <div style="width: 50%;">
        <img src="./media/southpark.png" alt="Southpark" style="width:80%; height:auto;">
    </div>
</div>


<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🤗 THANKS FOR YOUR ATTENTION 🤗</h1>
</div>

<center>
<img src="./media/thankyou.png" width="40%" height="40%" loop="false">
</center>

<br><br><br><br><br><br><br>

<center>
<img src="./media/questions.png" width="40%" height="40%" loop="false">
</center>